<a href="https://colab.research.google.com/github/Surabhi6300/Speech_Text_HuggingFace/blob/master/Hugging_face_Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers torchaudio torch

In [ ]:
from transformers import pipeline

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np

In [ ]:
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
from google.colab import output
from base64 import b64decode
import IPython.display as display

def record_audio(filename="my_audio.wav"):
    js = """
    async function record(sec) {
        const stream = await navigator.mediaDevices.getUserMedia({audio: true});
        const recorder = new MediaRecorder(stream);
        let data = [];
        recorder.ondataavailable = event => data.push(event.data);
        recorder.start();
        await new Promise(r => setTimeout(r, sec * 1000));
        recorder.stop();
        await new Promise(r => recorder.onstop = r);
        let audio = new Blob(data);
        let reader = new FileReader();
        reader.readAsDataURL(audio);
        reader.onloadend = () => {
            google.colab.kernel.invokeFunction('notebook.recorded', [reader.result], {});
        };
    }
    record(5)  // Record for 5 seconds
    """
    output.register_callback('notebook.recorded', lambda s: open(filename, "wb").write(b64decode(s.split(',')[1])))
    display.display(display.Javascript(js))

record_audio("my_audio.wav")


In [ ]:
import librosa

# Load the file that was saved
audio, sampling_rate = librosa.load("my_audio.wav", sr=16000)

print("Shape:", audio.shape)
print("Sample Rate:", sampling_rate)


In [ ]:
display.Audio('my_audio.wav', autoplay=True)

In [ ]:
input_values = processor(audio,sampling_rate=16000,return_tensors = "pt").input_values

input_values

In [ ]:
logits= model(input_values).logits
logits

In [ ]:
predicted_ids = torch.argmax(logits, dim = -1)

In [ ]:
transcriptions = processor.decode(predicted_ids[0])
transcriptions